In [47]:
from sage.crypto.sboxes import sboxes
from sage.all import *
from sboxU import * 
from sage.geometry.hyperplane_arrangement.affine_subspace import AffineSubspace
import itertools
import tracemalloc 
import gc
gc.enable()
F=GF(2)

In [2]:
XDDT=[]
YDDT=[]
S = list( sboxes['SKINNY_4']) 
DDT=ddt(S)
for a in range(0,16):
    lax=[]
    lay=[]
    for b in range(0,16): 
        lx=[]
        ly=[]
        for x in range(0,16): 
            if (int(S[(x^^a)%16])^^int(S[x]))==b : 
                lx.append(x)
                ly.append(S[x])
        lax.append(lx)
        lay.append(ly)
    XDDT.append(lax)
    YDDT.append(lay)
    
tran=[[] for i in range(16)]
traninv=[[] for i in range(16)]

for i in range(16):
    for j in range(16):
        if DDT[i][j]>0:
            tran[i].append(j)
            traninv[j].append(i)

traninv[3]

[2, 6, 7, 11, 14, 15]

In [3]:
def tobin(x, n):
    return [(x >> i) & 1 for i in reversed(range(0, n))]

def frombin(v):
    y = 0
    for i in range(0, len(v)):
        y = (y << 1) | int(v[i])
    return y

def apply_bin_mat(x, mat):
    n = mat.ncols()
    x = Matrix(GF(2), n, 1, tobin(x, n))
    y = mat * x
    return frombin(y.T[0][:mat.nrows()])

def bin_dot_product(alpha,beta,n): #alpha et beta sont des vecteurs binaires de longueur n
    res = 0
    for i in range (0,n):
        res =(res + alpha[i]*beta[i]) % 2
    return res
def from_64_to_16(x):
    res=[]
    for i in range(16):
        res.append(frombin(x[4*i:4*i+4]))
    return res
def ps(x,y,m,n): ## x and y are vectors of len n, each entry is m bits
    res=0
    for i in range(n):
        res += bin_dot_product(tobin(x[i],m),tobin(y[i],m),m)
    return res%2


In [4]:
## On précalcule l'orthogonal de tous les éléments de 0,15
F =GF(2)
Orthos=[[0] for i in range(16)] ## Dans Orthos les éléments sont des nombres entre 0 et 15, dans Orthos_bin, des vecctor(F) 
Orthos[0]= [i for i in range(16)]
Orthos_bin=[[vector(F,[0,0,0,0])]for i in range(16)]
Orthos_bin[0]=[vector(F,tobin(i,4)) for i in range(16)]
for j in range(1,16):
    for i in range(1,16):
        if bin_dot_product(tobin(i,4), tobin(j,4),4)==0 :
            Orthos[j].append(i)
            Orthos_bin[j].append(vector(F,tobin(i,4)))
            


            
anti_Orthos_bin=[[] for i in range(16)]
for j in range(1,16):
    for i in range(1,16):
        if bin_dot_product(tobin(i,4), tobin(j,4),4)==1 :
            anti_Orthos_bin[j].append(vector(F,tobin(i,4)))
            
anti_Orthos_bin[1]   

[(0, 0, 0, 1),
 (0, 0, 1, 1),
 (0, 1, 0, 1),
 (0, 1, 1, 1),
 (1, 0, 0, 1),
 (1, 0, 1, 1),
 (1, 1, 0, 1),
 (1, 1, 1, 1)]

In [5]:
def donne_base(l): ## Donne une base d'un ev étant donnés ses éléments
    R=Matrix(F, l).echelon_form()
    return [R.row(i) for i in range(R.nrows()) if R.row(i).is_zero() == False]
v=vector(F,[1,0,0,1])
bases_Orthos=[donne_base(Orthos_bin[i]) for i in range(16)]

bases_Orthos[9]

[(1, 0, 0, 1), (0, 1, 0, 0), (0, 0, 1, 0)]

In [6]:
def compute_ortho(X): ## X est un ev de taille <=4 :
    V=span(bases_Orthos[X[0]])
    for i in range(1,len(X)):
        V=V.intersection(span(bases_Orthos[X[i]]))
    return V

def compute_anti_ortho(v):## Calcule l'anti ortho d'un élément v 
    return AffineSubspace(anti_Orthos_bin[v][0],compute_ortho([v]))

def donne_elts(A):## Donne la liste des éléments d'un espace affine A:
    l=A.linear_part().list()
    p=list(A.point())
    resultat = []
    for element in l:
        resultat.append(tuple([x + y for x, y in zip(element, p)]))
    return resultat

In [7]:
compute_anti_ortho(1)

Affine space p + W where:
  p = (0, 0, 0, 1)
  W = Vector space of degree 4 and dimension 3 over Finite Field of size 2
Basis matrix:
[1 0 0 0]
[0 1 0 0]
[0 0 1 0]

In [8]:
compute_anti_ortho(2).intersection(AffineSubspace(0,compute_ortho(XDDT[3][8])))

Affine space p + W where:
  p = (0, 1, 1, 1)
  W = Vector space of degree 4 and dimension 1 over Finite Field of size 2
Basis matrix:
[1 0 0 0]

In [9]:
## On cherche quand est-ce que XDDT(beta,gamma_1)ortho inter XDDT(beta, gamma4)ortho est inclus dans {0,4,8,12}
G=span([vector(F,[1,0,0,0]),vector(F,[0,1,0,0])])
charok=[]
for beta in range(16):
    for gamma1 in range(16):
        for gamma4 in range(16):
            if DDT[beta][gamma1] > 0 and DDT[beta][gamma4] > 0 :
                V=compute_ortho(XDDT[beta][gamma1]).intersection(compute_ortho(XDDT[beta][gamma4]))
                if V.dimension()>0 and V.is_subspace(G) :
                    charok.append((beta,gamma1,gamma4,V))
V=None


In [10]:
len(charok)

89

In [11]:
charokdim2=[t for t in charok if t[3].dimension()>1]
charokdim2

[(3,
  12,
  15,
  Vector space of degree 4 and dimension 2 over Finite Field of size 2
  Basis matrix:
  [1 0 0 0]
  [0 1 0 0]),
 (3,
  15,
  12,
  Vector space of degree 4 and dimension 2 over Finite Field of size 2
  Basis matrix:
  [1 0 0 0]
  [0 1 0 0]),
 (3,
  15,
  15,
  Vector space of degree 4 and dimension 2 over Finite Field of size 2
  Basis matrix:
  [1 0 0 0]
  [0 1 0 0])]

In [12]:
compute_ortho(XDDT[8][13]).intersection(compute_ortho(XDDT[8][12]))

Vector space of degree 4 and dimension 1 over Finite Field of size 2
Basis matrix:
[0 1 0 0]

In [13]:
compute_ortho(XDDT[3][12]).intersection(compute_ortho(XDDT[3][15]))

Vector space of degree 4 and dimension 2 over Finite Field of size 2
Basis matrix:
[1 0 0 0]
[0 1 0 0]

In [14]:
def somme(alpha,beta):
    res=0
    for x in range(16):
        if (bin_dot_product(alpha,tobin(x,4),4) + bin_dot_product(beta,tobin(S[x],4),4))%2==0:
            res +=1
        else : 
            res -=1
    return res

In [15]:
from sage.crypto.boolean_function import BooleanFunction
deg2components=[]
for i in range(16):
       if (BooleanFunction(component(i,S)).algebraic_degree())==2:
            deg2components.append(i)
deg2components

[4, 8, 12]

In [16]:
def computeLS(f):## Calcule LS pour f,composante de S de degré 2
    res=[[],[]]
    for a in range(16):
            ## On teste si la dérivée est constante:
            temp=f.derivative(a).algebraic_degree()
            if temp==-1 :
                res[0].append(a)
            if temp==0:
                res[1].append(a)
    return res

for i in deg2components:
    print(computeLS(BooleanFunction(component(i,S))))

[[0, 1], [8, 9]]
[[0, 2], [1, 3]]
[[0, 11], [1, 10]]


In [17]:
pos=[[] for i in range(len(deg2components))]
neg=[[] for i in range(len(deg2components))]

for i in range(len(deg2components)):
    temp =BooleanFunction(component(deg2components[i],S)).walsh_hadamard_transform()
    for j in range(16):
        if temp[j]>0:
            pos[i].append(j)
        if temp[j]<0:
            neg[i].append(j)
pos,neg

([[8], [1], [3, 7, 9]], [[10, 12, 14], [5, 9, 13], [13]])

In [18]:
deg2components

[4, 8, 12]

In [19]:
for i in deg2components:
    print(BooleanFunction(component(i,S)).walsh_hadamard_transform())

(0, 0, 0, 0, 0, 0, 0, 0, 8, 0, -8, 0, -8, 0, -8, 0)
(0, 8, 0, 0, 0, -8, 0, 0, 0, -8, 0, 0, 0, -8, 0, 0)
(0, 0, 0, 8, 0, 0, 0, 8, 0, 8, 0, 0, 0, -8, 0, 0)


In [20]:
## Vérifions que le support est bien celui prévu :
res=[]
for i in deg2components :
    LS=computeLS(BooleanFunction(component(i,S)))
    res.append(compute_anti_ortho(LS[1][0]).intersection(AffineSubspace(0,compute_ortho(LS[0]))))

[donne_elts(k) for k in res] 

[[(1, 0, 0, 0), (1, 1, 0, 0), (1, 0, 1, 0), (1, 1, 1, 0)],
 [(0, 0, 0, 1), (1, 0, 0, 1), (0, 1, 0, 1), (1, 1, 0, 1)],
 [(1, 0, 0, 1), (0, 0, 1, 1), (1, 1, 0, 1), (0, 1, 1, 1)]]

In [21]:
def reduced_walsh_coeff(X,U,Sv):
    res=0
    u=tobin(U,4)
    for x in X : 
        if (bin_dot_product(tobin(x,4),u,4) + Sv[x])%2==0:
            res+=1
        else :
            res -=1
    return res

def reduced_walsh_table(X,S):
    res=[[] for i in range(16)]
    for i in range(16):
        Sv=component(i,S)
        for u in range(16):
            res[i].append(reduced_walsh_coeff(X,u,Sv))
    return res
    


In [22]:
reduced_walsh_table(XDDT[1][10],S)

[[4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0],
 [4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4, 0],
 [0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4],
 [0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 4],
 [-4, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 0, -4, 0],
 [-4, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 0, -4, 0],
 [0, -4, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 0, -4],
 [0, -4, 0, -4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 0, -4],
 [0, 0, 0, 0, 0, -4, 0, -4, 0, -4, 0, -4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, -4, 0, -4, 0, -4, 0, -4, 0, 0, 0, 0],
 [0, 0, 0, 0, -4, 0, -4, 0, -4, 0, -4, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -4, 0, -4, 0, -4, 0, -4, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0],
 [0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0, 0]]

In [23]:
def ortho_dim2(alpha,beta):## Donne l'orthogonal de (alpha beta) en en calculant les éléments puis une base
    res=[]
    for x in Orthos_bin[alpha]:
        for y in Orthos_bin[beta]:
            res.append(vector(F, list(x)+list(y)))
    for x in anti_Orthos_bin[alpha]:
        for y in anti_Orthos_bin[beta]:
            res.append(vector(F, list(x)+list(y)))
    return span(donne_base(res))

    
    

In [24]:
ortho_dim2(1,2)

Vector space of degree 8 and dimension 7 over Finite Field of size 2
Basis matrix:
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 1 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 1]

In [25]:
bcd=[vector(F,[0,0,0,0,0,0,0,1]),vector(F,[0,0,0,0,0,0,1,0]),vector(F,[0,0,0,0,0,1,0,0]),vector(F,[0,0,0,0,1,0,0,0])]
bcg=[vector(F,[0,0,0,1,0,0,0,0]),vector(F,[0,0,1,0,0,0,0,0]),vector(F,[0,1,0,0,0,0,0,0]),vector(F,[1,0,0,0,0,0,0,0])]


In [26]:
def donne_x(alpha,beta):## DDT[alpha][beta]==4
    if DDT[alpha][beta]!=4:
        print("Mauvais cardinal dans donne_x")
        return()
    if XDDT[alpha][beta][0]^^XDDT[alpha][beta][1]==alpha:
        return XDDT[alpha][beta][0], XDDT[alpha][beta][2]
    else : 
        return XDDT[alpha][beta][0],XDDT[alpha][beta][1]

In [34]:
## On calcule T qui vit dans F⁸
def compute_T(theta,alpha,beta,gamma2):
    ## On prend l'intersection des différents sous-espaces, on concatène les bases des espaces qui vivent dans F⁴ avec
    ## [0,0,0,0]. Ne pas oublier d'ajouter les 4 vecteurs [0,t] avec t qui parcourt la bc de F⁴ 
    
                                   
    if DDT[alpha][beta]==4:
        x0,x1=donne_x(alpha,beta)
        return span([vector(F,list(b) + [0,0,0,0]) for b in compute_ortho(YDDT[theta][alpha]).basis()]+bcd
               ).intersection(span([vector(F,[0,0,0,0] + list(b)) for b in compute_ortho(XDDT[beta][gamma2]
                                                                                        ).basis() ]+bcg
                                  ).intersection(ortho_dim2(alpha,beta).intersection((ortho_dim2(x0^^x1,S[x0]^^S[x1])))))                                  
    else :
        return span([vector(F,list(b) + [0,0,0,0]) for b in compute_ortho(YDDT[theta][alpha]).basis()]+bcd
               ).intersection(span([vector(F,[0,0,0,0] + list(b)) for b in compute_ortho(XDDT[beta][gamma2]
                                                                                        ).basis() ]+bcg
                                  ).intersection(ortho_dim2(alpha,beta)))

T=compute_T(4,2,1,8)

In [28]:
tracemalloc.start()
snapshot1 = tracemalloc.take_snapshot()
compute_T(4, 2, 1, 8)
snapshot2 = tracemalloc.take_snapshot()
# Obtenez des statistiques sur les allocations mémoire

# Calculez la différence entre les deux instantanés
stats_diff = snapshot2.compare_to(snapshot1, 'lineno')

# Affichez les statistiques de différence
print("[ Difference Stats ]")
for stat in stats_diff:
    print(stat)

[ Difference Stats ]
/usr/lib/python3/dist-packages/sage/modules/free_module.py:847: size=8712 B (+8712 B), count=137 (+137), average=64 B
/usr/lib/python3/dist-packages/sage/modules/free_module.py:5890: size=5760 B (+5760 B), count=10 (+10), average=576 B
/usr/lib/python3/dist-packages/sage/modules/free_module.py:3978: size=4784 B (+4784 B), count=46 (+46), average=104 B
/usr/lib/python3/dist-packages/sage/modules/free_module.py:5886: size=4096 B (+4096 B), count=53 (+53), average=77 B
/usr/lib/python3/dist-packages/sage/matrix/matrix_space.py:925: size=2320 B (+2320 B), count=24 (+24), average=97 B
/usr/lib/python3/dist-packages/sage/categories/homset.py:651: size=1936 B (+1936 B), count=32 (+32), average=60 B
/usr/lib/python3.10/weakref.py:349: size=1360 B (+1360 B), count=17 (+17), average=80 B
/usr/lib/python3/dist-packages/sage/structure/sequence.py:430: size=1104 B (+1104 B), count=21 (+21), average=53 B
<frozen importlib._bootstrap>:1027: size=1024 B (+1024 B), count=2 (+2), av

In [36]:
def compute_V(b4,T):
    LS=computeLS(BooleanFunction(component(b4,S)))
    
    return AffineSubspace(list(compute_anti_ortho(LS[1][0]).intersection(AffineSubspace(0,compute_ortho(LS[0]))).point())+[0,0,0,0],span([vector(F,list(b) + [0,0,0,0])
                                  for b in compute_anti_ortho(LS[1][0]).intersection(AffineSubspace(0,compute_ortho(LS[0]))
                                        ).linear_part().basis()]+bcd)).intersection(AffineSubspace(0,T))
def compute_X(alpha,beta,gamma2):
    if DDT[alpha][beta]==2:
        return compute_ortho(XDDT[beta][gamma2]).intersection(compute_ortho([beta]))
    if DDT[alpha][beta]==4:
        x0,x1=donne_x(alpha,beta)
        return compute_ortho(XDDT[beta][gamma2]).intersection(
            compute_ortho([beta]).intersection(compute_ortho([S[x0]^^S[x1]])))

In [37]:
compute_V(8,T)

Affine space p + W where:
  p = (0, 1, 0, 1, 0, 0, 0, 0)
  W = Vector space of degree 8 and dimension 1 over Finite Field of size 2
Basis matrix:
[0 0 0 0 1 0 1 0]

In [31]:
def constante(alpha,beta,theta,gamma1,gamma2,gamma4):
    return (DDT[theta][alpha]*DDT[beta][gamma1]*DDT[beta][gamma2]*DDT[beta][gamma4]*DDT[alpha][beta])/(16**6)

In [45]:
def donne_borne(alpha,beta,theta,gamma1,gamma2,gamma4):## Valable uniquement si on a une charok
    C=constante(alpha,beta,theta,gamma1,gamma2,gamma4) 
    X=compute_X(alpha,beta,gamma2)
    T=compute_T(theta,alpha,beta,gamma2)
    somme=2**(X.dimension())
    for t in compute_ortho(XDDT[beta][gamma1]).intersection(compute_ortho(XDDT[beta][gamma4])).list():
        if span([t]).dimension()>0: ## Permet de tester t!=0
            V=compute_V(frombin(t),T)
            if V is not None : ## L'intersection de deux espaces affines peut être vide
                somme += 2**(V.dimension())
    gc.collect()
    return somme*C

    

In [50]:
gc.garbage

[]

In [51]:
res=[]
beta=1
gamma1=9
gamma4=9
alpha=6
for gamma2 in tran[beta]:
    for theta in traninv[alpha]:
        res.append(donne_borne(alpha,beta,theta,gamma1,gamma2,gamma4))
## Une éxécution de ce tye augmente la memoire de 2000 MiB

In [43]:
traninv[beta]

[2, 6, 7, 11, 14, 15]

In [50]:
##Calcul de la somme sur D1
f=component(4,S)
LS=computeLS(BooleanFunction(f))
res=[0 for a in range(16)]
for a in range(16):
    for u in range(16):
        if (u not in LS[0]) and (u not in LS[1]) :
            if (f[u]+bin_dot_product(tobin(u,4),tobin(a,4),4))%2==0:
                res[a]+=1
            else :
                res[a]-=1

res

[0, 0, 0, 0, 0, 0, 0, 0, 12, 0, -4, 0, -4, 0, -4, 0]

In [61]:
BooleanFunction(component(4,S)).algebraic_normal_form()


x1*x2 + x1 + x2 + x3 + 1

In [62]:
A=matrix(F,[[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,0]])
A.right_kernel()

Vector space of degree 4 and dimension 2 over Finite Field of size 2
Basis matrix:
[1 0 0 0]
[0 0 0 1]